# Group level statistics


## Retrieving First-Level results
As we now have the same contrast from multiple `subjects` we can define our `group level model`. At first, we need to gather the `individual contrast maps`: 

In [37]:
import os
from bids.layout import BIDSLayout

ds_path = 'FaceRecognition'
# Initialize the BIDS layout and include the derivatives in it
layout = BIDSLayout(os.path.join(ds_path, 'data/bids'), derivatives=True)
layout.add_derivatives(os.path.join(ds_path, "results", "first-level-2mm"))

We will collect individual t-maps that represent the BOLD activity estimate divided by the uncertainty about this estimate.

In [38]:
contrast = 'FacesScrambled'
stat_files = layout.get(desc = contrast, suffix='stat', extension = '.nii.gz')
print(*stat_files, sep = "\n")

<BIDSImageFile filename='/imaging/correia/dace/training/summer-school/COGNESTIC-fMRI/hands-on/FaceRecognition/results/first-level-2mm/sub-03/sub-03_task-facerecognition_desc-FacesScrambled_stat.nii.gz'>
<BIDSImageFile filename='/imaging/correia/dace/training/summer-school/COGNESTIC-fMRI/hands-on/FaceRecognition/results/first-level-2mm/sub-04/sub-04_task-facerecognition_desc-FacesScrambled_stat.nii.gz'>


`PyBIDS` returns unsorted subject list, that's a bit problematic. 

In [ ]:
print(layout.get_subjects())

In [ ]:
subjects = sorted(list(set([f.get_entities().get("subject") for f in stat_files])))
print(subjects)

## Displaying subject t-maps

In [ ]:
from nilearn import plotting
import matplotlib.pyplot as plt
from nilearn.glm.thresholding import threshold_stats_img

fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(14, 14))

for i, stat_map in enumerate(stat_files):
    cluster_map, threshold = threshold_stats_img(stat_map, alpha=.001, height_control='fpr', cluster_threshold=20)
    plotting.plot_glass_brain(stat_map.path, 
                              title = 'sub-' + subjects[i],
                              axes = axes[int(i / 4), int(i % 4)],
                              plot_abs = False, 
                              display_mode='x', 
                             threshold=threshold)
fig.suptitle(contrast + ' t-map (unc. p<.001, k=20)')
plotting.show()

## Estimate second level model

### Design matrix

The next step includes the definition of a `design matrix`. Here we will want to run a simple `one-sample t-test`. We just need to indicate as many `1` as we have subjects with first-level results.

In [ ]:
import pandas as pd
design_matrix = pd.DataFrame(
    [1] * len(stat_files),
    columns=['intercept'])
design_matrix

### Model specification and fit

In [ ]:
from nilearn.glm.second_level import SecondLevelModel

second_level_model = SecondLevelModel(smoothing_fwhm = 8.0)
second_level_model = second_level_model.fit(
    stat_files,
    design_matrix = design_matrix)

### Contrast estimation

In [ ]:
z_map = second_level_model.compute_contrast(output_type='z_score')

### Thresholding and plotting

In [ ]:
cluster_map, threshold = threshold_stats_img(
    z_map, alpha=0.001, 
    height_control='fpr', 
    cluster_threshold=20)

from nilearn.datasets import load_mni152_template
template = load_mni152_template()
print('Uncorrected p<.001 threshold: %.3f' % threshold)
plotting.plot_stat_map(
    cluster_map, 
    threshold = threshold,       
    display_mode = 'ortho',
   cut_coords = [37,-84,-8],
    black_bg = True,
    bg_img = template,
    title = contrast + ' (unc. p<.001, k=20)')

fig = plt.gcf()
fig.set_size_inches(10,3)
plt.show()

In [ ]:
from nilearn.reporting import get_clusters_table
get_clusters_table(z_map, threshold, cluster_threshold=20, two_sided=False, min_distance=8.0)

In [ ]:
"""
from nilearn.reporting import make_glm_report

report = make_glm_report(model = second_level_model,
                         contrasts = 'intercept',
                         threshold = 3,
                         cluster_threshold = 30,
                         display_mode = 'ortho'
                         )

report
"""

In [ ]:
# atlasreader https://github.com/miykael/atlasreader

## Second level for multiple contrasts

In [ ]:
import warnings;
warnings.filterwarnings('ignore');

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# ======================================================================
# Dace Apšvalka (MRC CBU 2022)
# First level fMRI analysis using Nilearn
# ======================================================================

# ======================================================================
# IMPORT RELEVANT PACKAGES
# ======================================================================
import os
import numpy as np
import pandas as pd
from bids.layout import BIDSLayout
from nilearn.datasets import load_mni152_template
from nilearn.image import resample_to_img
from nilearn.glm.second_level import SecondLevelModel
from nilearn.reporting import get_clusters_table
from atlasreader import create_output

# ======================================================================
# DEFINE PATHS
# ======================================================================
ds_path = 'FaceRecognition'
outdir = os.path.join(ds_path, 'results', 'group-level')
if not os.path.exists(outdir):
    os.makedirs(outdir)

# ======================================================================
# WHICH CONTRASTS
# ======================================================================
contrasts = {'FamousUnfamiliar': 'Famous > Unfamiliar',
             'UnfamiliarFamous': 'Unfamiliar > Famous',
             'FacesScrambled': 'Faces > Scrambled',
             'ScrambledFaces': 'Scrambled > Faces',
             'EffectsOfInterest': 'Effects of interest'}

# ======================================================================
# PREPARE OTHER SUFF
# ======================================================================

# Initialize the BIDS layout and include the derivatives in it
layout = BIDSLayout(os.path.join(ds_path, 'data/bids'), derivatives=True)
layout.add_derivatives(os.path.join(ds_path, "results", "first-level-2mm"))

# load a template to resample images to if needed
template = load_mni152_template()

# ======================================================================
# PERFORM GROUP LEVEL ANALYSIS PER CONTRAST
# ======================================================================

for contrast_id, contrast_val in contrasts.items():
    stat_files = layout.get(desc = contrast_id, suffix = 'stat', extension = '.nii.gz')
    result_name = 'group_zmap_' + contrast_id + '_unc001k20'
      
    design_matrix = pd.DataFrame([1] * len(stat_files),
                                 columns=['intercept'])
    
    second_level_model = SecondLevelModel(smoothing_fwhm = 8.0)
    second_level_model = second_level_model.fit(
        stat_files,
        design_matrix = design_matrix)
    
    z_map = second_level_model.compute_contrast(output_type='z_score')
        
    # get threshold
    cluster_map, threshold = threshold_stats_img(z_map, alpha=.05, height_control='fpr', cluster_threshold=20)
    # get peak clusters    
    peaks = get_clusters_table(z_map, stat_threshold=threshold, cluster_threshold=20)
    
    # if there are significant voxels, then save the img and the plot
    try: 
        peak_xyz = peaks.loc[0, ['X','Y','Z']]
        # create plot
        plotting.plot_stat_map(
            cluster_map,
            threshold = threshold, 
            display_mode='ortho',
            cut_coords = peak_xyz, 
            black_bg = True, 
            title = contrast_val + ' unc. p<.001, k=20'
        )
        
        plt.show()
        # save results                   
        z_map.to_filename(os.path.join(outdir, result_name + '.nii.gz'))
    except KeyError:
        print('\t', contrast_val, 'has no significant voxels.')      
    
    # generate and save also atlasreader output
    create_output(
        os.path.join(outdir, result_name + '.nii.gz'), 
        cluster_extent = 20, 
        voxel_thresh = threshold,
        outdir = os.path.join(outdir, 'atlasreader', contrast_id)
    )